## Libraries

In [1]:
from time import time
import sys
import os
import glob
import csv

from six.moves import xrange
import tensorflow as tf
import numpy as np
import Image

import math

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

DATA_PATH = '../data/statefarmchallenge/'

## Convert to recoreds

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _byte_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def convert(images, labels, name):
    """Convert images and labesl to TFRecord"""
    num_examples = labels.shape[0]
    if images.shape[0] != num_examples:
        raise ValueError("images size {} did not match labels size {}"
                         .format(images.shape[0], num_examples))
        
    rows = images.shape[1]
    cols = images.shape[2]
    depth = images.shape[3]

    filename = os.path.join(DATA_PATH, name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(labels[index]),
            'image_raw': _byte_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()
       

def load_image(infilename):
    """Load our image and return as numpy array"""
    img = Image.open(infilename)
    data = np.asarray(img)
    return data


def load(rows, data_path):
    """Load a batch of images & labels for converting"""
    images = []
    labels = []
    
    print('Loading batch')
    for row in rows:
        path = os.path.join(data_path, row[1], row[2])
        data = load_image(path)
        images.append(data)
        labels.append(int(row[1][1]))
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels



def gen_record(size, records, name, num_batches, data_path):
    """Generate a record file by batching data"""
    examples_per_batch = size // num_batches
    
    if examples_per_batch <= 1:
        raise ValueError('Batch size must be greater than 1')
        
    print("Generating " + name + " record")
    for i in xrange(num_batches):
        k = i * examples_per_batch
        record_b = records[k : k + examples_per_batch]

        images_b, labels_b = load(record_b, data_path)

        convert(images_b, labels_b, name)
                
    # What about left overs???
    records_processed = examples_per_batch * num_batches
    left_over = size - records_processed
    
    if left_over > 0:
        record_b = records[records_processed:]
        images_b, labels_b = load(record_b, data_path)
        convert(images_b, labels_b, name)
        
        

def read_csv(path):
    """Read CSV record and return as list"""
    print('Reading csv')
                
    with open(path, 'rb') as f:
        reader = csv.reader(f)
        records = list(reader)
    
    return records
        

    
def main(argv):
    records = read_csv(DATA_PATH + 'driver_imgs_list.csv')
    num_examples = len(records)
    
    # Generate validation TFRecords (20%)
    validation_size = num_examples // 5
    val_records = records[ : validation_size]
    gen_record(size=validation_size, 
               records=val_records, 
               name='validation', 
               num_batches=5, 
               data_path=DATA_PATH + 'imgs/train/')

    
    # Generate training TFRecords
    train_size = num_examples - validation_size
    train_records = records[validation_size : ]
    gen_record(size=train_size, 
               records=train_records, 
               name='train', 
               num_batches=10, 
               data_path=DATA_PATH + 'imgs/train')
                


if __name__ == '__main__':
    tf.app.run()

Reading csv
Generating validation record
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Generating train record
Loading batch


## Input Pipeline 

In [1]:
TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'validation.tfrecords'

batch_size = 256
epochs = 2
height = 480
width = 640
depth = 3
image_size = height * width * depth

    
def input_pipeline():
    # Create queue for csv file
    file_queue = tf.train.string_input_producer([DATA_FILE_PATH], num_epochs=num_epochs)

    image, label = decode_and_process_img(file_queue)

    images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, 
                                            num_threads=6,
                                            capacity=1000 + 3 * batch_size,
                                            min_after_dequeue=1000)

    return image_batch, label_batch



def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
            'height' : tf.FixedLenFe,
            'width' :,
            'depth' :,
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
      })

    # Convert from a scalar string to unit8 tensor
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    tf.reshape(image, [height, width, depth])

    # OPTIONAL: Could reshape image and apply distortions or whitening here


    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)

    return image, label

    
def inputs(train, batch_size, num_epochs):
    if not num_epochs: num_epochs = None
    filename = os.path.join(DATA_PATH,
                          TRAIN_FILE if train else VALIDATION_FILE)

    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer(
            [filename], num_epochs=num_epochs)

    # Even when reading in multiple threads, share the filename
    # queue.
    image, label = read_and_decode(filename_queue)

    # Shuffle the examples and collect them into batch_size batches.
    # (Internally uses a RandomShuffleQueue.)
    # We run this in two threads to avoid being a bottleneck.
    images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size,
        # Ensures a minimum amount of shuffling of examples.
        min_after_dequeue=1000)

    return images, sparse_labels

IndentationError: expected an indented block (<ipython-input-1-62efe6f51668>, line 22)

Use tf records to run our model as a threaded queue

## Helper Functions

In [ ]:
def _dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot


def _variabel_on_cpu(name, shape, initializer, regularizer):
    """Helper to create a Variable stored on CPU memory.""""
    with tf.device('\cpu0:'):
        var = tf.get_variable(name, shape, initializer=initializer, regularizer=regularizer)
    
    return var



def _variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
     with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
   
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)

    
def local_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = _variable_with_weight_decay([input_dim, output_dim])
            variable_summaries(weights, layer_name + '/weights')
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases, layer_name + '/biases')
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.histogram_summary(layer_name + '/pre_activations', preactivate)
            
    activations = act(preactivate, 'activation')
    tf.histogram_summary(layer_name + '/activations', activations)
    return activations


def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = _variabel_on_cpu("weights", kernel_shape,
        initializer=tf.uniform_unit_scaling_initializer(factor=2.0 / kernel_shape[0]),
        regularizer=tf.contrib.layers.l2_regularizer(.001))
    
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

TODO: 
 - add images to summary
 - change learning rate to exp step
 - examine learning 
 - clean up layers with funtions

## Hyper parameters

In [ ]:
NUM_CLASSES = 10

# Convolution output depths 
depth_1 = 32
depth_2 = 64
depth_3 = 64

# Kernel shapes
patch_size = 10
num_channels = 3

# Local layer size
num_hidden = 2048


learning_rate = .01


## Model

In [2]:
def inference(images, keep_prob):
    """
    Args:
    images: Images placeholder, from inputs().
    Returns:
    softmax_linear: Output tensor with the computed logits.
    """

    # Conv1 /w pooling
    with tf.name_scope('conv1') as scope:
        conv1 = conv_relu(images, [patch_size, patch_size, channels, depth_1], [depth1])     
        pool1 = tf.nn.avg_pool(conv1, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME')
        norm1 = tf.nn.local_response_normalization(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

        
    # Conv2 /w pooling 
    with tf.name_scope('conv2') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth1, depth2], stddev=0.01))
        biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
        conv = tf.nn.conv2d(norm1, weights, [1, 1, 1, 1], padding='SAME', name='conv2')
        conv2 = tf.nn.relu(conv, name=scope)
    
    pool2 = tf.nn.avg_pool(conv2, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool2')
    norm2 = tf.nn.local_response_normalization(pool2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    
    # Conv3 /w pooling
    with tf.name_scope('conv3') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth2, depth3], stddev=0.01))
        biases = tf.Variable(tf.constant(1.0, shape=[depth3]))
        conv = tf.nn.conv2d(norm2, weights, [1, 1, 1, 1], padding='SAME', name='conv3')
        conv3 = tf.nn.relu(conv, name=scope)
    
    pool3 = tf.nn.avg_pool(conv3, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool3')
    norm3 = tf.nn.local_response_normalization(pool3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm3') 
    shape = norm3.get_shape().as_list()
    reshape = tf.reshape(norm3, [shape[0], shape[1] * shape[2] * shape[3]])
        
        
    #local4
    with tf.name_scope('local3') as scope:
        weights = tf.Variable(
            tf.truncated_normal([image_size, num_hidden],
                                stddev=1.0 / math.sqrt(float(num_hidden))))

        biases = tf.Variable(tf.zeros([num_hidden]))
        z = tf.matmul(reshape, weights) + biases
        local4 = tf.nn.relu(z, name=scope)
        drop = tf.nn.dropout(local4, keep_prob)

    # local5
    with tf.name_scope('local4') as scope:
        weights = tf.Variable(
            tf.truncated_normal([num_hidden, num_hidden],
                                stddev=1.0 / math.sqrt(float(num_hidden))), name='weights_5')

        biases = tf.Variable(tf.zeros([num_hidden]), name='biases_5')
        z = tf.matmul(drop, weights) + biases
        local5 = tf.nn.relu(z, name=scope)
        drop = tf.nn.dropout(local5, keep_prob)
        
        
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([num_hidden, NUM_CLASSES],
                                stddev=1.0 / math.sqrt(float(num_hidden))), name='weights_s')

        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases_s')
        logits = tf.matmul(drop, weights) + biases
        
    return logits


def loss(logits, labels):
    """"
    Calculates the loss from the logits and the labels.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int64 - [batch_size].
    Returns:
    loss: Loss tensor of type float.
    """
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss


def training(loss, learning_rate):
    """
    Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.
    Returns:
    train_op: The Op for training.
    """
    # Add a scalar summary for the snapshot loss. (Tensor board)
    tf.scalar_summary('loss', loss)

    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # decay on our learning rate as we progress with training
#     TODO: Use step for learn rate
    learn_rate = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.8)
    tf.scalar_summary('learn rate', learning_rate)
    
    # Create the gradient Adam optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op


def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
    Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
    """
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))


## Train

In [ ]:
def run_training():
    tf.reset_default_graph()
    with tf.Graph().as_default():
        
        keep_prob = tf.placeholder(tf.float32)
        
        images, labels = inputs(True, batch_size, num_epochs)
        
        logits = inference(images, keep_prob)
                
        loss_op = loss(logits, labels)
        
        train_op = training(loss_op, learning_rate)

        init_op = tf.initialize_all_variables()


        sess = tf.Session()
        sess.as_default()
        
        sess.run(init_op)

        # Start populating the filename queue.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        

    try:
        step = 0
        while not coord.should_stop():
            start_time = time.time()

            _, loss_value = sess.run([train_op, loss_op], feed_dict={keep_prob : 0.5})

            duration = time.time() - start_time

            # Print an overview fairly often.
            if step % 100 == 0:
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                
            elif step % 1000 == 0:
                # Save model
                
            step += 1
    
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (epochs, step))
    
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

        
    # Wait for threads to finish.
    coord.join(threads)
    sess.close()
        
        
run_training()

## Test 

In [ ]:
def load_test_images(path):
    """Load image to classify"""
    test_images = []
    paths = os.path.join(path, '*.jpg')
    image_files = glob.glob(paths)
    for i in paths:
        image = Image.open(i)
        test_images.append(image.getdata())
    
